## 1. Import


In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from tqdm import tqdm


## 2. 평가 함수 정의


In [2]:
def _validate_input(answer_df, submission_df):
    # ① 컬럼 개수·이름 일치 여부
    if len(answer_df.columns) != len(submission_df.columns) or not all(answer_df.columns == submission_df.columns):
        raise ValueError("The columns of the answer and submission dataframes do not match.")
    
    # ② 필수 컬럼에 NaN 존재 여부
    if submission_df.isnull().values.any():
        raise ValueError("The submission dataframe contains missing values.")
    
    # ③ pair 중복 여부
    pairs = list(zip(submission_df["leading_item_id"], submission_df["following_item_id"]))
    if len(pairs) != len(set(pairs)):
        raise ValueError("The submission dataframe contains duplicate (leading_item_id, following_item_id) pairs.")

def comovement_f1(answer_df, submission_df):
    """공행성쌍 F1 계산"""
    ans = answer_df[["leading_item_id", "following_item_id"]].copy()
    sub = submission_df[["leading_item_id", "following_item_id"]].copy()
    ans["pair"] = list(zip(ans["leading_item_id"], ans["following_item_id"]))
    sub["pair"] = list(zip(sub["leading_item_id"], sub["following_item_id"]))
    
    G = set(ans["pair"])
    P = set(sub["pair"])
    
    tp = len(G & P)
    fp = len(P - G)
    fn = len(G - P)
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0
    
    return f1

def comovement_nmae(answer_df, submission_df, eps=1e-6):
    """
    전체 U = G ∪ P에 대한 clipped NMAE 계산
    """
    ans = answer_df[["leading_item_id", "following_item_id", "value"]].copy()
    sub = submission_df[["leading_item_id", "following_item_id", "value"]].copy()
    ans["pair"] = list(zip(ans["leading_item_id"], ans["following_item_id"]))
    sub["pair"] = list(zip(sub["leading_item_id"], sub["following_item_id"]))
    
    G = set(ans["pair"])
    P = set(sub["pair"])
    U = G | P
    
    ans_val = dict(zip(ans["pair"], ans["value"]))
    sub_val = dict(zip(sub["pair"], sub["value"]))
    
    errors = []
    for pair in U:
        if pair in G and pair in P:
            # 정수 변환(반올림)
            y_true = int(round(float(ans_val[pair])))
            y_pred = int(round(float(sub_val[pair])))
            rel_err = abs(y_true - y_pred) / (abs(y_true) + eps)
            rel_err = min(rel_err, 1.0)  # 오차 100% 이상은 100%로 간주
        else:
            rel_err = 1.0  # FN, FP는 오차 100%
        errors.append(rel_err)
    
    return np.mean(errors) if errors else 1.0

def comovement_score(answer_df, submission_df):
    _validate_input(answer_df, submission_df)
    S1 = comovement_f1(answer_df, submission_df)
    nmae_full = comovement_nmae(answer_df, submission_df, 1e-6)
    S2 = 1 - nmae_full
    score = 0.6 * S1 + 0.4 * S2
    return score


## 3. 데이터 전처리 및 학습/검증 분리


In [3]:
train = pd.read_csv('../data/raw/train.csv')

# year, month, item_id 기준으로 value 합산 (seq만 다르다면 value 합산)
monthly = (
    train
    .groupby(["item_id", "year", "month"], as_index=False)["value"]
    .sum()
)

# year, month를 하나의 키(ym)로 묶기
monthly["ym"] = pd.to_datetime(
    monthly["year"].astype(str) + "-" + monthly["month"].astype(str).str.zfill(2)
)

# item_id × ym 피벗 (월별 총 무역량 매트릭스 생성)
pivot = (
    monthly
    .pivot(index="item_id", columns="ym", values="value")
    .fillna(0.0)
)

# 2025-07-01을 기준으로 학습/검증 분리
val_date = pd.to_datetime("2025-07-01")

# 학습 데이터: 2025-07-01 이전 데이터만 사용
pivot_train = pivot.loc[:, pivot.columns < val_date].copy()
print(f"학습 데이터 기간: {pivot_train.columns.min()} ~ {pivot_train.columns.max()}")
print(f"학습 데이터 shape: {pivot_train.shape}")

# 검증 데이터: 2025-07-01 데이터
if val_date in pivot.columns:
    print(f"\n검증 데이터 날짜: {val_date}")
    print(f"검증 데이터 shape: ({pivot.shape[0]}, 1)")
else:
    print(f"\n경고: {val_date} 데이터가 pivot에 없습니다.")


학습 데이터 기간: 2022-01-01 00:00:00 ~ 2025-06-01 00:00:00
학습 데이터 shape: (100, 42)

검증 데이터 날짜: 2025-07-01 00:00:00
검증 데이터 shape: (100, 1)


## 4. 공행성쌍 탐색 (학습 데이터만 사용)


In [4]:
def safe_corr(x, y):
    if np.std(x) == 0 or np.std(y) == 0:
        return 0.0
    return float(np.corrcoef(x, y)[0, 1])

def find_comovement_pairs(
    pivot, 
    max_lag=6, 
    min_nonzero=12, 
    corr_threshold=0.4
):
    """
    Parameters
    ----------
    pivot : pd.DataFrame
        item_id별 월별 거래량을 행렬로 표현한 데이터프레임입니다. (index: item_id, columns: ym)

    max_lag : int, default=6
        리더 품목(leading)과 팔로워 품목(following) 간 최대 탐색 시차(lag, 개월). 
        - lag=1이면 리더의 변동이 1개월 뒤 팔로워에 영향을 주는지 확인합니다.
        - lag=2,3,...,k까지 값을 늘릴수록 더 오래된 영향/지연 효과 패턴도 포함하게 됩니다.
        - 예를 들어 max_lag=6이면 최대 6개월까지 지연 반응을 모두 검토합니다.
        - max_lag 값이 크면 잠재적 패턴을 더 많이 탐색할 수 있지만, 너무 크면 표본이 부족해질 수 있고 실제적인 의미가 약화될 수 있습니다.
        - 업무적으로는 3~6 사이의 값을 가장 흔히 사용합니다(월별 데이터에서 한 분기~반년 내 영향이 있을 가능성이 높기 때문).

    min_nonzero : int, default=12
        한 품목이 최소한으로 비제로(0이 아닌) 거래량을 기록한 월 개수.
        - min_nonzero 값이 너무 작으면 빈약한 시계열이 공행성 판별에 노이즈를 많이 줄 수 있고,
        - 너무 크면 데이터가 부족해져 유의미한 쌍을 찾지 못할 수 있습니다.
        - 월 단위라면 최소 1년(12개월) 정도를 기본으로 많이 사용합니다.

    corr_threshold : float, default=0.4
        피어슨 상관계수의 임계값(절댓값 기준).
        - 이 값보다 크거나 같은 경우만 공행성쌍으로 간주.
        - 절댓값이므로 양의 상관관계(동일 방향)뿐 아니라 음의 상관관계(반대 방향)의 강한 패턴도 반영됩니다.
        - 보통 0.3~0.4 이상이면 실무적으로 '상당한 경향성'이 있다고 인정합니다.
        - 0.1~0.2: 민감하게 적용되며, 잡음이 많이 포함될 수 있습니다.
        - 0.4~0.6: 신뢰성과 검출 건수의 균형. 실무에서 가장 추천됨.
        - 0.7 이상: 매우 강한 상관성만 허용, 선정 쌍이 급격히 줄어듦(보수적으로 참고).
        - 실무에서는 0.4~0.6(대부분 0.4 또는 0.5)을 많이 사용합니다.
          그 이유는 실제 시장 데이터는 노이즈와 예외적 상황이 많아 '완벽한' 상관계수를 드물게 보이기 때문에,
          0.4 이상이면 경향성이 명확하다고 판단하는 경우가 많습니다.
    """
    items = pivot.index.to_list()
    months = pivot.columns.to_list()
    n_months = len(months)

    results = []

    for i, leader in tqdm(enumerate(items)):
        x = pivot.loc[leader].values.astype(float)
        if np.count_nonzero(x) < min_nonzero:
            continue

        for follower in items:
            if follower == leader:
                continue

            y = pivot.loc[follower].values.astype(float)
            if np.count_nonzero(y) < min_nonzero:
                continue

            best_lag = None
            best_corr = 0.0

            # lag = 1 ~ max_lag 탐색
            for lag in range(1, max_lag + 1):
                if n_months <= lag:
                    continue
                corr = safe_corr(x[:-lag], y[lag:])
                if abs(corr) > abs(best_corr):
                    best_corr = corr
                    best_lag = lag

            # 임계값 이상이면 공행성쌍으로 채택
            if best_lag is not None and abs(best_corr) >= corr_threshold:
                results.append({
                    "leading_item_id": leader,
                    "following_item_id": follower,
                    "best_lag": best_lag,
                    "max_corr": best_corr,
                })

    pairs = pd.DataFrame(results)
    return pairs

# 학습 데이터로만 공행성쌍 탐색
pairs = find_comovement_pairs(pivot_train)
print("탐색된 공행성쌍 수:", len(pairs))
pairs.head()


100it [00:12,  7.99it/s]

탐색된 공행성쌍 수: 1453


,leading_item_id,following_item_id,best_lag,max_corr
0,AANGBULD,APQGTRMF,5,-0.459240
1,AANGBULD,DEWLVASR,6,0.673163
2,AANGBULD,DNMPSKTB,4,-0.434721
3,AANGBULD,EVBVXETX,6,0.453442
4,AANGBULD,FTSVTTSR,3,0.533976


## 5. 회귀 모델 학습 (학습 데이터만 사용)


In [5]:
def build_training_data(pivot, pairs):
    """
    공행성쌍 + 시계열을 이용해 (X, y) 학습 데이터를 만드는 함수
    input X:
      - b_t, b_t_1, a_t_lag, max_corr, best_lag
    target y:
      - b_t_plus_1
    """
    months = pivot.columns.to_list()
    n_months = len(months)

    rows = []

    for row in pairs.itertuples(index=False):
        leader = row.leading_item_id
        follower = row.following_item_id
        lag = int(row.best_lag)
        corr = float(row.max_corr)

        if leader not in pivot.index or follower not in pivot.index:
            continue

        a_series = pivot.loc[leader].values.astype(float)
        b_series = pivot.loc[follower].values.astype(float)

        # t+1이 존재하고, t-lag >= 0인 구간만 학습에 사용
        for t in range(max(lag, 1), n_months - 1):
            b_t = b_series[t]
            b_t_1 = b_series[t - 1]
            a_t_lag = a_series[t - lag]
            b_t_plus_1 = b_series[t + 1]

            rows.append({
                "b_t": b_t,
                "b_t_1": b_t_1,
                "a_t_lag": a_t_lag,
                "max_corr": corr,
                "best_lag": float(lag),
                "target": b_t_plus_1,
            })

    df_train = pd.DataFrame(rows)
    return df_train

df_train_model = build_training_data(pivot_train, pairs)
print('생성된 학습 데이터의 shape :', df_train_model.shape)
df_train_model.head()


생성된 학습 데이터의 shape : (54376, 6)


,b_t,b_t_1,a_t_lag,max_corr,best_lag,target
0,582317.0,539873.0,14276.0,-0.45924,5.0,759980.0
1,759980.0,582317.0,52347.0,-0.45924,5.0,216019.0
2,216019.0,759980.0,53549.0,-0.45924,5.0,537693.0
3,537693.0,216019.0,0.0,-0.45924,5.0,205326.0
4,205326.0,537693.0,26997.0,-0.45924,5.0,169440.0


In [6]:
# 회귀모델 학습
feature_cols = ['b_t', 'b_t_1', 'a_t_lag', 'max_corr', 'best_lag']

train_X = df_train_model[feature_cols].values
train_y = df_train_model["target"].values

reg = LinearRegression()
reg.fit(train_X, train_y)
print("모델 학습 완료")


모델 학습 완료


## 6. 검증 데이터(2025-07-01)에 대한 예측


In [7]:
def predict(pivot, pairs, reg, target_date):
    """
    특정 날짜에 대한 예측 수행
    pivot: 전체 pivot 데이터 (검증 날짜 이전까지의 데이터를 사용)
    pairs: 공행성쌍
    reg: 학습된 회귀 모델
    target_date: 예측할 날짜 (pd.Timestamp)
    
    주의: 예측 시점에서는 target_date의 실제 값을 모르므로,
    target_date 이전까지의 데이터만 사용하여 예측합니다.
    """
    # target_date 이전까지의 데이터만 사용 (예측 시점에서는 target_date의 실제 값을 모름)
    pivot_for_pred = pivot.loc[:, pivot.columns < target_date].copy()
    
    if len(pivot_for_pred.columns) == 0:
        raise ValueError(f"target_date {target_date} 이전의 데이터가 없습니다.")
    
    months = pivot_for_pred.columns.to_list()
    n_months = len(months)
    
    # 가장 마지막 달 index (target_date 직전 달, 즉 2025-06-01)
    t_last = n_months - 1
    t_prev = t_last - 1 if t_last > 0 else t_last

    preds = []

    for row in tqdm(pairs.itertuples(index=False)):
        leader = row.leading_item_id
        follower = row.following_item_id
        lag = int(row.best_lag)
        corr = float(row.max_corr)

        if leader not in pivot_for_pred.index or follower not in pivot_for_pred.index:
            continue

        a_series = pivot_for_pred.loc[leader].values.astype(float)
        b_series = pivot_for_pred.loc[follower].values.astype(float)

        # t_last - lag 가 0 이상인 경우만 예측
        if t_last - lag < 0:
            continue

        # b_t: 가장 최근 달의 값 (target_date 직전 달, 즉 2025-06-01)
        # b_t_1: 그 이전 달의 값 (2025-05-01)
        # a_t_lag: leader의 lag개월 전 값
        b_t = b_series[t_last]
        b_t_1 = b_series[t_prev] if t_prev >= 0 else 0.0
        a_t_lag = a_series[t_last - lag]

        X_test = np.array([[b_t, b_t_1, a_t_lag, corr, float(lag)]])
        y_pred = reg.predict(X_test)[0]

        # (후처리 1) 음수 예측 → 0으로 변환
        # (후처리 2) 소수점 → 정수 변환 (무역량은 정수 단위)
        y_pred = max(0.0, float(y_pred))
        y_pred = int(round(y_pred))

        preds.append({
            "leading_item_id": leader,
            "following_item_id": follower,
            "value": y_pred,
        })

    df_pred = pd.DataFrame(preds)
    return df_pred

# 2025-07-01 이전까지의 데이터를 사용하여 2025-07-01 예측
submission = predict(pivot, pairs, reg, val_date)
print(f"\n생성된 예측 결과 수: {len(submission)}")
submission.head()


1453it [00:00, 4272.52it/s]


생성된 예측 결과 수: 1453


,leading_item_id,following_item_id,value
0,AANGBULD,APQGTRMF,328596
1,AANGBULD,DEWLVASR,495116
2,AANGBULD,DNMPSKTB,5447518
3,AANGBULD,EVBVXETX,4993057
4,AANGBULD,FTSVTTSR,293583


## 7. 정답 데이터 생성 (2025-07-01 실제 값)


In [8]:
# train.csv에서 2025-07-01의 실제 데이터 추출
val_year = val_date.year
val_month = val_date.month

answer_raw = train[
    (train["year"] == val_year) & 
    (train["month"] == val_month)
].copy()

# 공행성쌍에 해당하는 (leading_item_id, following_item_id) 쌍만 추출
# 정답 데이터는 실제로 존재하는 모든 쌍이 아니라, 우리가 예측한 쌍에 대한 정답만 필요
# 하지만 평가 함수는 전체 U = G ∪ P에 대해 계산하므로, 실제 존재하는 모든 쌍을 포함해야 함

# 실제 데이터에서 leading_item_id와 following_item_id 쌍을 어떻게 구성할지 결정
# baseline 코드를 보면 공행성쌍 탐색 결과에 따라 예측하므로,
# 정답도 같은 구조로 만들어야 함

# 실제로는 정답 데이터가 어떤 형식인지 확인이 필요하지만,
# 여기서는 train.csv에서 2025-07-01의 모든 item_id에 대한 value를 추출하고,
# 공행성쌍 구조에 맞춰 정답을 생성

# 방법 1: 실제 train.csv에서 2025-07-01의 모든 item_id별 value 합산
answer_monthly = (
    answer_raw
    .groupby("item_id", as_index=False)["value"]
    .sum()
)

# 공행성쌍에 대해 정답 생성
# 실제로는 정답 데이터가 어떤 형식인지에 따라 달라지지만,
# 여기서는 공행성쌍의 following_item_id에 대한 value를 정답으로 사용
answer_dict = dict(zip(answer_monthly["item_id"], answer_monthly["value"]))

answer_list = []
for row in pairs.itertuples(index=False):
    follower = row.following_item_id
    if follower in answer_dict:
        answer_list.append({
            "leading_item_id": row.leading_item_id,
            "following_item_id": follower,
            "value": answer_dict[follower]
        })
    else:
        # 해당 item_id가 2025-07-01에 데이터가 없는 경우 0으로 처리
        answer_list.append({
            "leading_item_id": row.leading_item_id,
            "following_item_id": follower,
            "value": 0
        })

answer_df = pd.DataFrame(answer_list)
print(f"생성된 정답 데이터 수: {len(answer_df)}")
print(f"정답 데이터 value 합계: {answer_df['value'].sum():,.0f}")
answer_df.head()


생성된 정답 데이터 수: 1453
정답 데이터 value 합계: 7,009,633,460


,leading_item_id,following_item_id,value
0,AANGBULD,APQGTRMF,40608.0
1,AANGBULD,DEWLVASR,482787.0
2,AANGBULD,DNMPSKTB,4507669.0
3,AANGBULD,EVBVXETX,5061099.0
4,AANGBULD,FTSVTTSR,246916.0


## 8. 평가 점수 계산


In [9]:
# 평가 수행
try:
    score = comovement_score(answer_df, submission)
    f1 = comovement_f1(answer_df, submission)
    nmae = comovement_nmae(answer_df, submission)
    
    print(f"=== 2025-07-01 검증 결과 ===")
    print(f"F1 Score (S1): {f1:.6f}")
    print(f"NMAE: {nmae:.6f}")
    print(f"NMAE Score (S2 = 1 - NMAE): {1 - nmae:.6f}")
    print(f"최종 Score (0.6 * S1 + 0.4 * S2): {score:.6f}")
except Exception as e:
    print(f"평가 중 오류 발생: {e}")
    print("\n정답 데이터와 제출 데이터 비교:")
    print(f"정답 데이터 shape: {answer_df.shape}")
    print(f"제출 데이터 shape: {submission.shape}")
    print(f"\n정답 데이터 컬럼: {answer_df.columns.tolist()}")
    print(f"제출 데이터 컬럼: {submission.columns.tolist()}")


=== 2025-07-01 검증 결과 ===
F1 Score (S1): 1.000000
NMAE: 0.578910
NMAE Score (S2 = 1 - NMAE): 0.421090
최종 Score (0.6 * S1 + 0.4 * S2): 0.768436
